# 客户生命周期价值 (CLV) 计算
> By SherryAGI | 预测客户未来价值

## CLV 公式

```
CLV = (平均订单价值 × 购买频率 × 毛利率) × 客户生命周期

CLV = (AOV × F × GM) × T
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# SherryAGI - CLV Analysis
print('🚀 客户生命周期价值分析 by SherryAGI')

## 1. 基础 CLV 计算

In [ ]:
# 模拟订单数据
np.random.seed(42)
n_orders = 5000

orders = pd.DataFrame({
    'customer_id': np.random.randint(1, 501, n_orders),
    'order_date': pd.date_range('2024-01-01', periods=n_orders, freq='2H'),
    'amount': np.random.exponential(80, n_orders),
    'cost': np.random.exponential(50, n_orders)
})

orders['profit'] = orders['amount'] - orders['cost']
orders.head()

In [ ]:
# 计算每个客户的指标
customer_stats = orders.groupby('customer_id').agg({
    'amount': ['mean', 'sum', 'count'],
    'profit': 'sum',
    'order_date': ['min', 'max']
})

customer_stats.columns = ['avg_order_value', 'total_revenue', 'order_count', 
                          'total_profit', 'first_order', 'last_order']

# 计算客户生命周期（天）
customer_stats['lifespan_days'] = (customer_stats['last_order'] - customer_stats['first_order']).dt.days + 1

# 计算购买频率（每月）
customer_stats['purchase_freq'] = customer_stats['order_count'] / (customer_stats['lifespan_days'] / 30)

customer_stats.head(10)

## 2. 简单 CLV 计算

In [ ]:
# 假设平均客户生命周期 24 个月
avg_lifespan_months = 24

# 毛利率
gross_margin = orders['profit'].sum() / orders['amount'].sum()
print(f'毛利率: {gross_margin:.2%}')

# 计算 CLV
customer_stats['CLV'] = (
    customer_stats['avg_order_value'] * 
    customer_stats['purchase_freq'] * 
    gross_margin * 
    avg_lifespan_months
)

customer_stats[['avg_order_value', 'purchase_freq', 'CLV']].describe()

## 3. CLV 分布可视化

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CLV 分布
customer_stats['CLV'].hist(bins=50, ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_title('CLV 分布 by SherryAGI')
axes[0].set_xlabel('CLV (¥)')
axes[0].set_ylabel('客户数')
axes[0].axvline(customer_stats['CLV'].median(), color='red', linestyle='--', label=f'中位数: ¥{customer_stats["CLV"].median():.0f}')
axes[0].legend()

# CLV vs 订单数
axes[1].scatter(customer_stats['order_count'], customer_stats['CLV'], alpha=0.5)
axes[1].set_title('CLV vs 订单数')
axes[1].set_xlabel('订单数')
axes[1].set_ylabel('CLV (¥)')

plt.tight_layout()
plt.show()

## 4. CLV 客户分层

In [ ]:
# 按 CLV 分层
customer_stats['CLV_tier'] = pd.qcut(
    customer_stats['CLV'], 
    q=4, 
    labels=['🥉 低价值', '🥈 中等价值', '🥇 高价值', '💎 顶级价值']
)

# 各层统计
tier_summary = customer_stats.groupby('CLV_tier').agg({
    'CLV': ['count', 'mean', 'sum'],
    'total_revenue': 'sum'
})

tier_summary.columns = ['客户数', '平均CLV', 'CLV总和', '总收入']
tier_summary['收入占比'] = tier_summary['总收入'] / tier_summary['总收入'].sum() * 100

tier_summary

## 5. 业务洞察

| 指标 | 应用 |
|------|------|
| 平均 CLV | 设定获客成本上限 (CAC < CLV) |
| CLV 分布 | 识别高价值客户群 |
| CLV/CAC 比率 | 评估营销 ROI（目标 > 3:1）|

In [ ]:
avg_clv = customer_stats['CLV'].mean()
print(f'📊 平均 CLV: ¥{avg_clv:.2f}')
print(f'💰 建议获客成本上限: ¥{avg_clv/3:.2f} (CLV/3)')
print(f'🎯 顶级客户 CLV: ¥{customer_stats[customer_stats["CLV_tier"]=="💎 顶级价值"]["CLV"].mean():.2f}')

---
**By SherryAGI** | [DigitalTransformationAI](https://github.com/AIB612/DigitalTransformationAI)